In [2]:
from sources.utils.sanitize_and_transliterate import sanitize_and_transliterate
from sources.normalizer.news_normalizer import NewsNormalizer
from sources.saver.file_saver import FileSaver
from sources.annotator.xml_annotator import XMLAnnotator
from sources.source.suspilne_source import SuspilneSource
from sources.parser.suspilne_parser import SuspilneParser

corpus_path = "corpus"

for record in SuspilneSource.records():
    title = record.metadata.title
    safe_title = sanitize_and_transliterate(title)
    
    print(f"Processing: {title}")
    content = record.content()

    annotator = XMLAnnotator()
    document = SuspilneParser.parse(content, metadata=record.metadata,annotator=annotator)
    base_text = annotator.get_string(document)
    
    saver = FileSaver(path=f"{corpus_path}/base")
    saver.save(base_text, name=f"{safe_title}.txt")
    
    warnings = []
    errors = []
    
    counter = 1
    def normalize(text: str) -> (str, str):
        global counter
        
        new_text, new_warnings, new_errors = NewsNormalizer.normalize(text)
        
        if new_errors or new_warnings:
            id = f"e{counter}"
            counter += 1
        
            warnings.append((id, new_warnings))
            errors.append((id, new_errors))
            
            return new_text, id
        
        return new_text, None
    
    document = annotator.update_elements(document, normalize)
    
    annotator.add_warnings(document, warnings)
    annotator.add_errors(document, errors)
    normalized_text = annotator.get_string(document)

    saver = FileSaver(path=f"{corpus_path}/normalized")
    saver.save(normalized_text, name=f"{safe_title}.txt")
    
    
    print(f"Finished: {title}")





Processing: Голова ЦРУ: "українці воюватимуть голими руками, якщо не матимуть умов для міцного миру"
Finished: Голова ЦРУ: "українці воюватимуть голими руками, якщо не матимуть умов для міцного миру"
Processing: Умєров: Готові організувати технічну зустріч щодо імплементації домовленостей в Ер-Ріяді
Finished: Умєров: Готові організувати технічну зустріч щодо імплементації домовленостей в Ер-Ріяді
Processing: Третя збірна кваліфікувалася на ЧС-2026 з футболу: хто вийшов від Азії
Finished: Третя збірна кваліфікувалася на ЧС-2026 з футболу: хто вийшов від Азії
Processing: У Харкові невідомий стріляв по автобусах: поліція почала перевірку
Finished: У Харкові невідомий стріляв по автобусах: поліція почала перевірку
Processing: Затримали підозрюваного у підготовці теракту, спогади про Кирила Ульмана. Головні новини Дніпра та області 25 березня
Finished: Затримали підозрюваного у підготовці теракту, спогади про Кирила Ульмана. Головні новини Дніпра та області 25 березня
Processing: Волц: "Роз